In [ ]:
# default_exp spectrogram_processor

# Spectrogram Processor

> Commandline program to generate spectrograms from audio data.

In [ ]:
#hide
from nbdev.showdoc import *

argparse [load filename ref](https://www.reddit.com/r/learnpython/comments/575rbb/confused_how_to_set_inputoutput_file_paths_with/)



In [ ]:
# export
import argparse
from pathlib import Path
import json
import librosa
import numpy as np
from PIL import Image
import time
from fastprogress import progress_bar

In [ ]:
# export
# ignore librosa pysoundfile load warning
import warnings
warnings.filterwarnings(
    action='ignore',
    category=UserWarning,
    module=r'librosa'
)

## Configuration

Get path to configuration file. *Note*: looks like commandline programs take path inputs relative to the console's address, not the program's.

In [ ]:
# export
parser = argparse.ArgumentParser()
parser.add_argument("--path", help="parameter filepath", nargs=1)
args = parser.parse_args()
# args = parser.parse_args(["--path", "parameters.json"])

In [ ]:
# export
fpath = Path(args.path[0])
assert fpath.exists(), (f"Filepath '{fpath}' not found.")
assert fpath.is_file(), (f"'{fpath}' is not a valid file.")

Edit and Save Parameters

In [ ]:
# hide
# # testing
# p = str((Path()/'sample_data').absolute())

# parameters = {
#     'n_fft':2048,
#     'hop_length':512,
#     'n_mels':256,
#     'mel_n_fft_override':None,
#     'mel_hop_length_override':None,
# #     'path':'/Users/jupyter/data',
#     'path':p,
#     'audio_folders':['train','test'],
# #     'audio_folders':['train'],
#     'codecs':['.flac']
# }
# # save
# with open('parameters.json', 'w') as file:
#     json.dump(parameters, file)

Load Parameters

In [ ]:
# export
with open(fpath, 'r') as file:
    parameters = json.load(file)
    print(f"Parameters file \"{fpath}\" loaded.")

Parameters file "parameters.json" loaded.


In [ ]:
# export
n_fft = parameters['n_fft']
hop_length = parameters['hop_length']
n_mels = parameters['n_mels']

mel_n_fft = parameters['mel_n_fft_override']
if mel_n_fft == None: mel_n_fft = n_fft

mel_hop_length = parameters['mel_hop_length_override']
if mel_hop_length == None: mel_hop_length = hop_length

if n_fft / n_mels < 2**3:
    print(f"Warning: N FFT ({n_fft}) is fewer than 3 powers of 2 greater than N Mels ({n_mels})."
          f" This may result in null values at lower mels in spectrograms.")

## Spectrograms

In [ ]:
# export
def compute_spectrogram(wf, n_fft=1024, hop_length=512):
    return librosa.power_to_db(np.abs(librosa.stft(wf, n_fft=n_fft, hop_length=hop_length))**2)
    
def compute_mel_spectrogram(wf, sr, n_fft=1024, hop_length=512, n_mels=128):
    return librosa.power_to_db(librosa.feature.melspectrogram(wf, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels))

def write_spectrogram(spgm, filepath):
    # convert fp32 array to 0-255 UInt8.
    shift = abs(spgm.min()) if spgm.min() < 0 else 0.0
    x = ((spgm + shift) * (255/(shift + spgm.max())))
    img = Image.fromarray(x.round().astype(np.uint8)[::-1]) # vertical flip
    img.save(filepath)

In [ ]:
# hide
# times = {'compute':[],'write':[]}

In [ ]:
# export
t = time.time()

In [ ]:
# export
path = Path(parameters['path'])
path_spgm_frq = path/'spectrogram_frq'
path_spgm_frq.mkdir(parents=True, exist_ok=True)
path_spgm_mel = path/'spectrogram_mel'
path_spgm_mel.mkdir(parents=True, exist_ok=True)

# get list of audio files
audio_files = []
for audio_folder in parameters['audio_folders']:
    audio_path = path/audio_folder
    audio_files += [file for file in audio_path.iterdir() if file.suffix in parameters['codecs']]
    
# load waveform
pb = progress_bar(audio_files)
for audio_file in pb:
    pb.comment = f"processing: {audio_file.parent.name}/{audio_file.name}"
    wf,sr = librosa.load(audio_file, sr=None)
    time.sleep(0.1)

    # compute freq & mel spectrograms
    spgm_frq = compute_spectrogram(wf, n_fft=n_fft, hop_length=hop_length)
    spgm_mel = compute_mel_spectrogram(wf, sr, n_fft=mel_n_fft, hop_length=mel_hop_length, n_mels=n_mels)

    # write spectrograms
    write_spectrogram(spgm_frq, path_spgm_frq/f"{audio_file.stem}.png")
    write_spectrogram(spgm_mel, path_spgm_mel/f"{audio_file.stem}.png")

In [ ]:
# export
print(f"\n{len(audio_files)} files processed in {time.strftime('%H:%M:%S', time.gmtime(time.time() - t))}")

In [ ]:
# hide
# # custom write
# t_compute = np.array(times['compute'])
# t_write = np.array(times['write'])
# estim = (t_compute.mean() + t_write.mean()) * (1992+4727)
# print(t_compute.sum(), t_compute.mean())
# print(t_write.sum(), t_write.mean())
# print(estim)
# print(spgm_frq.shape, spgm_mel.shape)
# time.strftime('%H:%M:%S', time.gmtime(estim))

## notebook export

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_spectrogram_processor.ipynb.
Converted experiments - spectrogram compression and timing statistics.ipynb.
Converted experiments - waveform analysis.ipynb.
Converted index.ipynb.


## asides

An alternate way to show progress bars. `progress_bar` is subordinated to a `master_bar` instance, `mb`. `mb` is a single-run loop, and used for writing comments. The only issue is this prints a default message "Epoch i/N" with each progress bar loop. If using only a `progress_bar`, you'll have to instatiate it in order to print comments/updates.

In [ ]:
# mb = master_bar(range(1))
# for itr in mb:
#     for audio_file in progress_bar(audio_files, parent=mb):
# #         mb.main_bar.comment = ""
#         mb.child.comment = f"processing: {audio_file.name}"
# #         mb.write(f"")
# #         print(f"processing: {audio_file.name}")
#         time.sleep(0.1)